In [3]:
! unzip 'All Midi Files.zip'

Archive:  All Midi Files.zip
   creating: All Midi Files/albeniz/
  inflating: All Midi Files/albeniz/alb_esp1.mid  
  inflating: All Midi Files/albeniz/alb_esp2.mid  
  inflating: All Midi Files/albeniz/alb_esp3.mid  
  inflating: All Midi Files/albeniz/alb_esp4.mid  
  inflating: All Midi Files/albeniz/alb_esp5.mid  
  inflating: All Midi Files/albeniz/alb_esp6.mid  
  inflating: All Midi Files/albeniz/alb_se1.mid  
  inflating: All Midi Files/albeniz/alb_se2.mid  
  inflating: All Midi Files/albeniz/alb_se3.mid  
  inflating: All Midi Files/albeniz/alb_se4.mid  
  inflating: All Midi Files/albeniz/alb_se5.mid  
  inflating: All Midi Files/albeniz/alb_se6.mid  
  inflating: All Midi Files/albeniz/alb_se7.mid  
  inflating: All Midi Files/albeniz/alb_se8.mid  
   creating: All Midi Files/bach/
  inflating: All Midi Files/bach/bach_846.mid  
  inflating: All Midi Files/bach/bach_847.mid  
  inflating: All Midi Files/bach/bach_850.mid  
   creating: All Midi Files/balakir/
  inflating: 

# Importing

In [4]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model 
from sklearn.model_selection import train_test_split

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

# Data Preparation

In [6]:
def read_files(file):
    notes=[]
    notes_to_parse=None
    #parse the midi file
    midi=converter.parse(file)
    #seperate all instruments from the file
    instrmt=instrument.partitionByInstrument(midi)
    for part in instrmt.parts:
        #fetch data only of Piano instrument
        if 'Piano' in str(part):
            notes_to_parse=part.recurse()
            for element in notes_to_parse:
                if type(element)==note.Note:
                    notes.append(str(element.pitch))
                elif type(element)==chord.Chord:
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes
#retrieve paths recursively from inside the directories/files
file_path=['schumann'] 
all_files=glob.glob('/content/All Midi Files/'+ file_path[0]+'/*.mid',recursive=True)
notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)])

100%|██████████| 24/24 [00:52<00:00,  2.17s/it]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [7]:
#unique notes
notess = sum(notes_array,[]) 
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))

#get the threshold frequency
print("\nFrequency notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

Unique Notes: 218

Frequency notes
30 : 111
50 : 87
70 : 70
90 : 65


In [8]:
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes 
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

x_new=np.array(x) 
y_new=np.array(y)

In [9]:
#reshape input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#split the input and value into training and testing sets
#80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

# Model Buiding

In [10]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [12]:
#create the model
model = Sequential()
#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 87)                22359     
                                                                 
Total params: 877,655
Trainable params: 877,655
Non-trai

In [13]:
#compile the model using Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#train the model on training sets and validate on testing sets
with tf.device('/gpu:0'):
    model.fit(
        x_train,y_train,
        batch_size=128,epochs=80, 
        validation_data=(x_test,y_test))

Epoch 1/80
117/117 [==============================] - 11s 25ms/step - loss: 4.1517 - accuracy: 0.0411 - val_loss: 4.0942 - val_accuracy: 0.0415
Epoch 2/80
117/117 [==============================] - 2s 17ms/step - loss: 4.0954 - accuracy: 0.0439 - val_loss: 4.0543 - val_accuracy: 0.0458
Epoch 3/80
117/117 [==============================] - 2s 18ms/step - loss: 4.0404 - accuracy: 0.0480 - val_loss: 3.9994 - val_accuracy: 0.0490
Epoch 4/80
117/117 [==============================] - 2s 18ms/step - loss: 3.9805 - accuracy: 0.0558 - val_loss: 3.9306 - val_accuracy: 0.0589
Epoch 5/80
117/117 [==============================] - 2s 19ms/step - loss: 3.9100 - accuracy: 0.0672 - val_loss: 3.8794 - val_accuracy: 0.0720
Epoch 6/80
117/117 [==============================] - 2s 19ms/step - loss: 3.8448 - accuracy: 0.0723 - val_loss: 3.8303 - val_accuracy: 0.0733
Epoch 7/80
117/117 [==============================] - 2s 18ms/step - loss: 3.7680 - accuracy: 0.0867 - val_loss: 3.7834 - val_accuracy: 0.083

In [14]:
model.save("MOD")

INFO:tensorflow:Assets written to: MOD/assets


INFO:tensorflow:Assets written to: MOD/assets


In [15]:
model=load_model("MOD")
#generate random index
index = np.random.randint(0,len(x_test)-1)
#get the data of generated index from x_test
music_pattern = x_test[index]

out_pred=[] #it will store predicted notes

#iterate till 200 note is generated
for i in range(200):

  #reshape the music pattern 
  music_pattern = music_pattern.reshape(1,len(music_pattern),1)
  
  #get the maximum probability value from the predicted output
  pred_index = np.argmax(model.predict(music_pattern))
  #get the note using predicted index and
  #append to the output prediction list
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)
  
  #update the music pattern with one timestep ahead
  music_pattern = music_pattern[1:]

In [19]:
output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        #cast the current note to Note object and
        #append the current note 
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
    
    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up 
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
  
  else:
    #cast the pattern to Note object apply the offset and 
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_composed_music.mid')

'AI_composed_music.mid'